In [4]:
from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from datetime import datetime, timezone
from dotenv import load_dotenv
import os
load_dotenv()

# Initialize Graphiti
graphiti = Graphiti("bolt://localhost:7687", os.environ["NEO4J_USER"], os.environ["NEO4J_PASSWORD"])

# Initialize the graph database with Graphiti's indices. This only needs to be done once.
graphiti.build_indices_and_constraints()


<coroutine object Graphiti.build_indices_and_constraints at 0x7db1aa5335a0>

In [5]:
# Add episodes
episodes = [
    "Kamala Harris is the Attorney General of California. She was previously "
    "the district attorney for San Francisco.",
    "As AG, Harris was in office from January 3, 2011 – January 3, 2017",
]
for i, episode in enumerate(episodes):
    await graphiti.add_episode(
        name=f"Freakonomics Radio {i}",
        episode_body=episode,
        source=EpisodeType.text,
        source_description="podcast",
        group_id="1",
        reference_time=datetime.now(timezone.utc)
    )

# Search the graph
# Execute a hybrid search combining semantic similarity and BM25 retrieval
# Results are combined and reranked using Reciprocal Rank Fusion
results = await graphiti.search('Who was the California Attorney General?')

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: content)} {position: line: 4, column: 18, offset: 143} for query: '\n        MATCH (e:Episodic) WHERE e.valid_at <= $reference_time \n        AND ($group_ids IS NULL) OR e.group_id in $group_ids\n        RETURN e.content AS content,\n            e.created_at AS created_at,\n            e.valid_at AS valid_at,\n            e.uuid AS uuid,\n            e.group_id AS group_id,\n            e.name AS name,\n            e.source_description AS source_description,\n            e.source AS source\n        ORDER BY e.created_at DESC\n        LIMIT $num_episodes

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such fulltext schema index: node_name_and_summary}

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: name_embedding)} {position: line: 4, column: 48, offset: 120} for query: '\n            MATCH (n:Entity)\n            WHERE n.group_id IN $group_ids\n            WITH n, vector.similarity.cosine(n.name_embedding, $search_vector) AS score\n            WHERE score > $min_score\n            RETURN\n                n.uuid As uuid,\n                n.group_id AS group_id,\n                n.name AS name, \n                n.name_embedding AS name_embedding,\n                n.created_at AS created_at, \n                n.summary AS summary\n            ORDER

In [3]:
# print(results)
# [
#     EntityEdge(
# │   uuid = '3133258f738e487383f07b04e15d4ac0',
# │   source_node_uuid = '2a85789b318d4e418050506879906e62',
# │   target_node_uuid = 'baf7781f445945989d6e4f927f881556',
# │   created_at = datetime.datetime(2024, 8, 26, 13, 13, 24, 861097),
# │   name = 'HELD_POSITION',
# # the fact reflects the updated state that Harris is
# # no longer the AG of California
# │   fact = 'Kamala Harris was the Attorney General of California',
# │   fact_embedding = [
# │   │   -0.009955154731869698,
# │       ...
# │   │   0.00784289836883545
# │],
# │   episodes = ['b43e98ad0a904088a76c67985caecc22'],
# │   expired_at = datetime.datetime(2024, 8, 26, 20, 18, 1, 53812),
# # These dates represent the date this edge was true.
# │   valid_at = datetime.datetime(2011, 1, 3, 0, 0, tzinfo= < UTC >),
# │   invalid_at = datetime.datetime(2017, 1, 3, 0, 0, tzinfo= < UTC >)
# )
# ]

# Rerank search results based on graph distance
# Provide a node UUID to prioritize results closer to that node in the graph.
# Results are weighted by their proximity, with distant edges receiving lower scores.
results = await graphiti.search('Who was the California Attorney General?', center_node_uuid)
print(results)

# Close the connection
graphiti.close()

NameError: name 'center_node_uuid' is not defined